In [18]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v0").unwrapped

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

In [20]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        #self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(128, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        #output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = Categorical(F.softmax(output, dim=-1))
        return distribution

In [21]:
class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        #self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(128, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        #output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

In [22]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

In [25]:
def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters())
    optimizerC = optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            #env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor.forward(state), critic.forward(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor, 'actor.pkl')
    torch.save(critic, 'critic.pkl')
    env.close()

In [27]:
if __name__ == '__main__':
    if os.path.exists('model/actor.pkl'):
        actor = torch.load('model/actor.pkl')
        print('Actor Model loaded')
    else:
        actor = Actor(state_size, action_size).to(device)
    if os.path.exists('model/critic.pkl'):
        critic = torch.load('model/critic.pkl')
        print('Critic Model loaded')
    else:
        critic = Critic(state_size, action_size).to(device)
    trainIters(actor, critic, n_iters=500)

Iteration: 0, Score: 10
Iteration: 1, Score: 12
Iteration: 2, Score: 12
Iteration: 3, Score: 13
Iteration: 4, Score: 13
Iteration: 5, Score: 14
Iteration: 6, Score: 13
Iteration: 7, Score: 9
Iteration: 8, Score: 17
Iteration: 9, Score: 14
Iteration: 10, Score: 13
Iteration: 11, Score: 13
Iteration: 12, Score: 11
Iteration: 13, Score: 33
Iteration: 14, Score: 17
Iteration: 15, Score: 24
Iteration: 16, Score: 11
Iteration: 17, Score: 29
Iteration: 18, Score: 21
Iteration: 19, Score: 14
Iteration: 20, Score: 10
Iteration: 21, Score: 8
Iteration: 22, Score: 20
Iteration: 23, Score: 13
Iteration: 24, Score: 29
Iteration: 25, Score: 19
Iteration: 26, Score: 17
Iteration: 27, Score: 15
Iteration: 28, Score: 18
Iteration: 29, Score: 31
Iteration: 30, Score: 10
Iteration: 31, Score: 12
Iteration: 32, Score: 9
Iteration: 33, Score: 14
Iteration: 34, Score: 15
Iteration: 35, Score: 31
Iteration: 36, Score: 15
Iteration: 37, Score: 11
Iteration: 38, Score: 14
Iteration: 39, Score: 18
Iteration: 40

Iteration: 320, Score: 158
Iteration: 321, Score: 202
Iteration: 322, Score: 155
Iteration: 323, Score: 217
Iteration: 324, Score: 78
Iteration: 325, Score: 205
Iteration: 326, Score: 255
Iteration: 327, Score: 199
Iteration: 328, Score: 143
Iteration: 329, Score: 174
Iteration: 330, Score: 205
Iteration: 331, Score: 186
Iteration: 332, Score: 198
Iteration: 333, Score: 154
Iteration: 334, Score: 214
Iteration: 335, Score: 172
Iteration: 336, Score: 180
Iteration: 337, Score: 452
Iteration: 338, Score: 114
Iteration: 339, Score: 218
Iteration: 340, Score: 473
Iteration: 341, Score: 415
Iteration: 342, Score: 196
Iteration: 343, Score: 359
Iteration: 344, Score: 321
Iteration: 345, Score: 271
Iteration: 346, Score: 343
Iteration: 347, Score: 294
Iteration: 348, Score: 138
Iteration: 349, Score: 176
Iteration: 350, Score: 156
Iteration: 351, Score: 339
Iteration: 352, Score: 335
Iteration: 353, Score: 397
Iteration: 354, Score: 162
Iteration: 355, Score: 387
Iteration: 356, Score: 199
It

In [ ]:
env.close()